# Examp Minggu 5

app.ipynb (FastAPI)

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
import model.py
import database
import requests
import datetime

# app.ipynb (FastAPI)

class PropertyInput(BaseModel):
 bedrooms: int
 bathrooms: int
 sqft: float
 location: str

@app.post("/predict")
def predict_price(property: PropertyInput):

    """
        menerima data properti dari pengguna dan mengembalikan prediksi harga 
        Args:
            property (PropertyInput): Model yang berisi informasi properti
        Returns:
            dict: {"user_id": str, "predicted_price": float}
            - user_id: ID pengguna yang didapat dari API publik.
            - predicted_price: Harga rumah prediksi berdasarkan model.
    """

    # Ambil data dari API publik
    user_data = requests.get("https://randomuser.me/api/").json()
    user_id = user_data['results'][0]['login']['uuid']
    
    # Prediksi harga rumah
    predicted_price = model.predict(property.bedrooms, property.bathrooms, 
    property.sqft, property.location)
    
    # Simpan ke MongoDB
    database.save_to_db(user_id, property.dict(), predicted_price)
 
    return {"user_id": user_id, "predicted_price": predicted_price}

@app.get("/stats")
def get_stats():
    """
    Mengembalikan statistik sederhana tentang dataset harga rumah (mean, median, dll).

    Returns:
        dict: {"mean": float, "median": float, "std_dev": float}
            - mean: Harga rata-rata rumah dalam dataset.
            - median: Harga tengah rumah dalam dataset.
            - std_dev: Standar deviasi harga rumah dalam dataset.
    """
    # Ambil statistik dataset
    stats = model.get_statistics()
    return stats




In [ ]:
# model.ipynb (Model Machine Learning)
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
# Load dataset harga rumah
data = pd.read_csv("housing.csv")
X = data[["bedrooms", "bathrooms", "sqft", "location"]]
y = data["price"]

# Preprocessing
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2)

# Model
model = LinearRegression()
model.fit(X_train, y_train)
def predict(bedrooms, bathrooms, sqft, location):
 X_new = scaler.transform([[bedrooms, bathrooms, sqft, location]])
 return model.predict(X_new)[0]
def get_statistics():
 return {
 "mean": np.mean(y),
 "median": np.median(y),
 "std_dev": np.std(y)
 }

In [15]:
# database.ipynb (MongoDB Integration)
from pymongo import MongoClient
import datetime
client = MongoClient("mongodb://localhost:27017/")
db = client['housing_db']
def save_to_db(user_id, property_data, predicted_price):
 db.predictions.insert_one({
 "user_id": user_id,
 "property_data": property_data,
 "predicted_price": predicted_price,
 "timestamp": datetime.datetime.now()
 })
